In [1]:
import sys
import tweepy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from nltk import pos_tag
from collections import Counter 
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.naive_bayes import BernoulliNB


In [2]:
url_test = "datasets/corona_tweets_extracted.csv"
df_test_1 = pd.read_csv(url_test)
df_test = df_test_1[35000:55000].reset_index(drop=True)
print(df_test)

                  tweet_id                                            content  \
0      1240732695413395456  Any company that continues to deny their worke...   
1      1240732695459696642  The same Tolu Ogunlesi that insisted that Mr J...   
2      1240732695522664454  @pattonoswalt @AlanTudyk He can't pronounce co...   
3      1240732695644291073                 Aqui risking the corona en el work   
4      1240732695778324480  Day 5: We have rediscovered farming https://t....   
...                    ...                                                ...   
19995  1240863708219392001  this man’s piss could stop corona https://t.co...   
19996  1240863708290510849  #SocialDistancing #CoronaVirus#covid19#corona ...   
19997  1240863708588335104  There was a nasty flu that went round in Decem...   
19998  1240863708743487490  @YBRAP We will still be great even without the...   
19999  1240863708882046976  My housemate went for a walk and has been coug...   

       sentiment polarity  

In [13]:
url_test2 = "DataSets/group_datasets/corona_tweets_extracted_loc_date_test.csv"
df_test2 = pd.read_csv(url_test2)
df_test3 = df_test2[35000:45000].reset_index(drop=True)
print(df_test3)

                 tweet_id                                            content  \
0     1241623065756413957  @JoeBiden Bernie has raised over 2 million dol...   
1     1241623065785581569  When this corona shit is over please invite me...   
2     1241623065819148290         Corona virus sefty https://t.co/O7y0CP4BAS   
3     1241623065827524608  Money spent on Son/Daughter marriage by :-\n\n...   
4     1241623066213380098  #WhoCanSave_The_World\n\nIf you want to avoid ...   
...                   ...                                                ...   
9995  1241807261808254976  A Manipuri girl at Vijay Nagar, Delhi was spat...   
9996  1241807261951053827  When this corona shit is over please invite me...   
9997  1241807262013747200  now THIS is what I call high fashion https://t...   
9998  1241807262076690432  Foolish people of Indore celebration corona fe...   
9999  1241807262123020288  I got corona and an Std just from watching thi...   

      sentiment         location coordi

C:\Users\34722\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
url_train = "DataSets/group_datasets/corona_tweets_extracted_train.csv"
df_train = pd.read_csv(url_train)
df_train2 = df_train[35000:37000].reset_index(drop=True)
print(df_train2)

                 tweet_id                                            content  \
0     1240732695413395456  Any company that continues to deny their worke...   
1     1240732695459696642  The same Tolu Ogunlesi that insisted that Mr J...   
2     1240732695522664454  @pattonoswalt @AlanTudyk He can't pronounce co...   
3     1240732695644291073                 Aqui risking the corona en el work   
4     1240732695778324480  Day 5: We have rediscovered farming https://t....   
...                   ...                                                ...   
1995  1240732978357166081                           @yeeyi_1 I blame Corona.   
1996  1240732978415878145  here’s a deleted scene from TWD’s special on c...   
1997  1240732978441007112  😂😂😂 I love my country, but how did we get here...   
1998  1240732978487144449  When this Corona shit passes we have to promis...   
1999  1240732978529153025                  Dit mens. https://t.co/9Q53qYlJLj   

      sentiment polarity  
0     -0.238

In [27]:
url_corona = "DataSets/group_datasets/Covid300_ES_POLARITY100_classified4.csv"
df_corona = pd.read_csv(url_train)
df_corona2 = df_corona[35000:370000].reset_index(drop=True)
print(df_corona2)

                  tweet_id                                            content  \
0      1240732695413395456  Any company that continues to deny their worke...   
1      1240732695459696642  The same Tolu Ogunlesi that insisted that Mr J...   
2      1240732695522664454  @pattonoswalt @AlanTudyk He can't pronounce co...   
3      1240732695644291073                 Aqui risking the corona en el work   
4      1240732695778324480  Day 5: We have rediscovered farming https://t....   
...                    ...                                                ...   
77373  1241039751454416897  Shaheen Bagh is still on.\n\nMosques are open....   
77374  1241039751509049346  Over 50 Corona Virus cases in Maharashtra, hig...   
77375  1241039751551082497  Living in England during the Corona virus whil...   
77376  1241039751609815040  WHEN I HEARD KD HAD CORONA VIRUS😔 @nba YALL BE...   
77377  1241039752024948736  Liberal choreographer returns to Bengaluru fro...   

       sentiment polarity  

In [32]:
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re
df = pd.read_csv(url_test,encoding='utf-8')[35000:37000].reset_index(drop=True)
dftest = pd.read_csv(url_test2,encoding='utf-8')[35000:37000].reset_index(drop=True)
dftrain = pd.read_csv(url_train,encoding='utf-8')[35000:37000].reset_index(drop=True)
dfcorona = pd.read_csv(url_corona,encoding='utf-8').reset_index(drop=True)
#print(df)

## Preproceso
def frase(tw):
    fr = TextBlob(tw)
    return ' '.join(fr.words)


def importantes(tw):
    tlist = [a for a in tw.split() if a != 'user']
    clean = ' '.join([t for t in tlist if re.match(r'[^\W\d]*$',t)]) # quita caracteres raros y digitos
    clean = [w.lower() for w in clean.split() if w.lower() not in stopwords.words('english')]
    return clean

def quitadupl(twl):
  
    L = WordNetLemmatizer()
    fr = []
    for w in twl:
        txw = L.lemmatize(w,'v')
        fr.append(txw)
    return ' '.join(fr)



In [55]:
# Ejecutar solo la primera vezya que  tarda bastante
df['txt'] = df['content'].apply(lambda x : quitadupl(importantes(frase(x))))
df.to_pickle('preproc.pkl')

In [50]:
# Ejecutar solo la primera vezya que  tarda bastante
dftest['txt'] = dftest['content'].apply(lambda x : quitadupl(importantes(frase(x))))
dftest.to_pickle('preproc.pkl')

In [74]:
# Ejecutar solo la primera vezya que  tarda bastante
dftrain['txt'] = dftrain['content'].apply(lambda x : quitadupl(importantes(frase(x))))
dftrain.to_pickle('preproc.pkl')

In [33]:
# Ejecutar solo la primera vezya que  tarda bastante
dfcorona['txt'] = dfcorona['content'].apply(lambda x : quitadupl(importantes(frase(x))))
df.to_pickle('preproc.pkl')

In [6]:
df = pd.read_pickle('preproc.pkl')
print(df[0:5])

              tweet_id                                            content  \
0  1240732695413395456  Any company that continues to deny their worke...   
1  1240732695459696642  The same Tolu Ogunlesi that insisted that Mr J...   
2  1240732695522664454  @pattonoswalt @AlanTudyk He can't pronounce co...   
3  1240732695644291073                 Aqui risking the corona en el work   
4  1240732695778324480  Day 5: We have rediscovered farming https://t....   

   sentiment polarity                                                txt  
0  -0.238095      NEG  company continue deny workers pay sick leave d...  
1   0.000000      NEU  tolu ogunlesi insist mr jonathan must hold pre...  
2   0.000000      NEU         pattonoswalt alantudyk ca pronounce corona  
3   0.000000      NEU                        aqui risk corona en el work  
4   0.000000      NEU                          day rediscover farm https  


In [53]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
#Model Selection and Validation
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import CategoricalNB
from nltk.classify import MaxentClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score

pipeline = Pipeline([
    ('bow',CountVectorizer(analyzer='word')),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1))  
])

In [73]:
msg_train, msg_test, label_train, label_test = train_test_split(df['txt'], df['polarity'], test_size=0.2)
pipeline.fit(msg_train,label_train)
predictions = pipeline.predict(msg_test)
print(classification_report(predictions,label_test))
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))

              precision    recall  f1-score   support

         NEG       0.88      0.87      0.87       107
         NEU       0.90      0.85      0.87       211
         POS       0.75      0.85      0.80        82

    accuracy                           0.86       400
   macro avg       0.84      0.86      0.85       400
weighted avg       0.86      0.86      0.86       400

[[ 93  10   4]
 [ 12 180  19]
 [  1  11  70]]
0.8575


In [143]:
msg_train, msg_test, label_train, label_test = train_test_split(dftrain['txt'], dftrain['polarity'], test_size=0.2)
pipeline.fit(msg_train,label_train)
predictions = pipeline.predict(msg_test)
print(classification_report(predictions,label_test))
print(confusion_matrix(predictions,label_test))
                       
print(accuracy_score(predictions,label_test))

              precision    recall  f1-score   support

         NEG       0.83      0.87      0.85       103
         NEU       0.91      0.83      0.86       208
         POS       0.67      0.76      0.72        89

    accuracy                           0.82       400
   macro avg       0.80      0.82      0.81       400
weighted avg       0.83      0.82      0.83       400

[[ 90   5   8]
 [ 11 172  25]
 [  8  13  68]]
0.825


C:\Users\34722\Anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [149]:
msg_train, msg_test, label_train, label_test = train_test_split(dftest['txt'], dftrain['polarity'], test_size=0.2)
pipeline.fit(msg_train,label_train)
predictions = pipeline.predict(msg_test)
print(classification_report(predictions,label_test))
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))


              precision    recall  f1-score   support

         NEG       0.00      0.00      0.00         0
         NEU       1.00      0.48      0.65       400
         POS       0.00      0.00      0.00         0

    accuracy                           0.48       400
   macro avg       0.33      0.16      0.22       400
weighted avg       1.00      0.48      0.65       400

[[  0   0   0]
 [110 193  97]
 [  0   0   0]]
0.4825


C:\Users\34722\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
msg_train, msg_test, label_train, label_test = train_test_split(dfcorona['txt'], dfcorona['polarity'], test_size=0.2)
pipeline.fit(msg_train,label_train)
predictions = pipeline.predict(msg_test)
print(classification_report(predictions,label_test))
print(confusion_matrix(predictions,label_test))
print(accuracy_score(predictions,label_test))

              precision    recall  f1-score   support

         NEG       0.61      0.48      0.54        29
         NEU       0.77      0.65      0.71        37
         POS       0.00      0.00      0.00         0

    accuracy                           0.58        66
   macro avg       0.46      0.38      0.41        66
weighted avg       0.70      0.58      0.63        66

[[14  7  8]
 [ 9 24  4]
 [ 0  0  0]]
0.5757575757575758


C:\Users\34722\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
